In [2]:
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)

from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser

from typing import List, Tuple

import os
import json
import random

# Data

In [3]:
PATH = "..\\src\\data\\komus\\dataset.json"

with open(PATH, "r", encoding="UTF-8") as file:
    data = json.load(file)

len(data)

83755

# Utils

In [4]:
def create_model_prompts(system_prompt: str,
                         user_prompt: str) -> ChatPromptTemplate:
    system_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
    user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_prompt,
         user_prompt]
    )
    return chat_prompt

In [4]:
from difflib import ndiff

def highlight_diff(s1, s2):
    diff = ndiff(s1, s2)
    markdown_diff = ""
    
    for part in diff:
        if part.startswith('+'):
            markdown_diff += f'<span style="color: green;">{part[2:]}</span>'  # Добавленное
        elif part.startswith('-'):
            markdown_diff += f'<span style="color: red; text-decoration: line-through;">{part[2:]}</span>'  # Удалённое
        else:
            markdown_diff += part[2:]  # Общий текст
    
    return markdown_diff

# Models

## Gemini

In [ ]:
PROJECT_ID = 'axial-chemist-425510-p2'
LOCATION = "europe-west4"

# Укажите путь к вашему JSON-файлу с кредами
SERVICE_ACCOUNT_FILE = "../secrets/axial-chemist-425510-p2-b4eb1d622fbe.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_FILE


llm_params = {
    'model_name': 'gemini-2.0-flash-001',
    'project': PROJECT_ID,
    'location': LOCATION
}

llm = ChatVertexAI(**llm_params)

## Т-Pro

In [5]:
from langchain_openai import ChatOpenAI

T_PRO_CREDS = "../secrets/t-pro.json"

with open(T_PRO_CREDS) as file:
    model_params = json.load(file)

llm = ChatOpenAI(**model_params, temperature=0)

## Случайный выбор из подборки

In [6]:
idx = random.choices(list(range(len(data))), k = 100)

product_choices = [data[id] for id in idx]

# Аугментация

In [ ]:
SYSTEM_PROMPT = """
Проведи аугментацию товарной позиции по данным примерам.
Примеры:
{examples}

**Не создавай больше текста и уточнений, кроме ответа**
""".strip()

USER_PROMPT = """Проведи аугментацию для данной позиции {problem_title}"""

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [7]:
examples = [
    {
        "original_item": "4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota",
        "clean_item": "Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)"
    },
    {
        "original_item": "АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT",
        "clean_item": "Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт."
    },
        {
        "original_item": "Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют",
        "clean_item": "Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют "
    },
    {
        "original_item": "Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт",
        "clean_item": "Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)"
    }
]

examples_str = "\n".join([f"""
{i+1})Оригинальная товарная позиция: {product["clean_item"]}
Аугментированная товарная позиция: {product['original_item']}
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Оригинальная товарная позиция: Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)
Аугментированная товарная позиция: 4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota
2)Оригинальная товарная позиция: Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт.
Аугментированная товарная позиция: АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT
3)Оригинальная товарная позиция: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют 
Аугментированная товарная позиция: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют
4)Оригинальная товарная позиция: Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)
Аугментированная товарная позиция: Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт


## Запуск модели

In [ ]:
batch = [
    {
        "examples": examples_str,
        "problem_title": data[product_id]["title"]
    } for product_id in idx
]

[{'examples': '1)Оригинальная товарная позиция: Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)\nАугментированная товарная позиция: 4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota\n2)Оригинальная товарная позиция: Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт.\nАугментированная товарная позиция: АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT\n3)Оригинальная товарная позиция: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют \nАугментированная товарная позиция: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют\n4)Оригинальная товарная позиция: Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)\nАугментированная товарная позиция: Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт',
  'problem_title': 'Чайник электрический Scarlett SC-EK27G95 серебристый'},
 {'examples': '1)Оригинальная товарная позиция: Крышка главного тор

In [104]:
augment_chain = prompt | llm

res1 = augment_chain.batch(batch)
res2 = augment_chain.batch(batch)

In [137]:
# , highlight_diff(title, augmented_title.content)

print("\n\n".join(
        ["\n\n".join(
            ["\\\n".join([title, augmented_title.content]) for augmented_title in augmented_titles]
        ) for title, *augmented_titles in zip([data[id]["title"] for id in idx], res2)]
    )
)

Чайник электрический Scarlett SC-EK27G95 серебристый\
Эл.ЧАЙНИК SC-EK27G95 СЕР. SCARLETT

Лоток горизонтальный для бумаг Комус Art Deco пластиковый бирюзовый\
ЛОТОК ГОР. ДЛЯ БУМАГ ART DECO ПЛАСТ БИРЮЗОВЫЙ КОМУС

Топ для шкафа Torr (дуб каньон, 854x452x38 мм)\
Топ на шкаф TORR Дуб Каньон 854x452x38мм

Сапоги рабочие шахтерские мужские С192Т-МТЗ ПУ резиновые черные c композитным подноском размер 42\
С192Т-МТЗ САПОГИ РАБ/ШАХ МУЖ, ПУ, РЕЗИН, ЧЁРНЫЕ, КОМП. ПОДН., 42 РАЗМ.

Ложка столовая Luxstahl Lotus (кт3124) 20.4 см нержавеющая сталь (12 штук в упаковке)\
Ложка столовая LUXSTAHL LOTUS КТ3124, 20.4 СМ, НСТ, 12ШТ/УПАК

Жидкость для снятия лака EVI professional с ацетоном 1000 мл\
ЕВИ PROFESSIONAL ЖЕЛ.ДЛЯ СН.ЛАКА С АЦЕТ. 1000мл

Костюм рабочий зимний мужской з03-КПК с СОП красный/черный (размер 64-66, рост 170-176)\
Костюм рабочий зимний мужской Z03-КПК, СОП, красный/черный, 64-66, 170-176 размер/рост

Магнитола Ritmix RBB-100BT\
RITMIX RBB-100BT МАГНИТОЛА

Туфли мужские Алми Денис ПВХ из н

# Сравнение двух товарных позиций

In [7]:
class CompareListResponse(BaseModel):
    lost_fragments: List[str] = Field(..., description="Фрагменты строки, которые были потеряны")
    added_fragments: List[str] = Field(..., description="Фрагменты, которые были добавлены в аугментированную строку")

parser = PydanticOutputParser(pydantic_object=CompareListResponse)

SYSTEM_PROMPT = """
Дана оригинальная строка str1 и аугментированная строка str2. Необходимо:  
1. Найти фрагменты из str1, которые потерялись при аугментации (отсутствуют в str2).  
2. Найти фрагменты из str2, которые добавились при аугментации (отсутствуют в str1).    

Примеры:
{examples}

Правила:
- Если фрагмент сокращен при аугментации, то он не потерян.
- Уточнения, которые были созданы во время аугментации должны считаться, как добавление.
- Знаки препинания и спецсимволы не нужно рассматривать при анализе.
- В случае транслитерации правила транслитерации должны соблюдаться, в других случаях написание названий производителей или модели должно совпадать.
- Не учитывать отдельно предлоги.

В конце рассуждений напиши только окончательный ответ в формате: {format_instructions}.
""".strip()

USER_PROMPT = """
Оригинальная строка: {original_string}
Аугментированная строка: {augmented_string}

Сравни две эти строки.
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

## Примеры

In [8]:
examples = [
    {
        "original_string": "Батарея ExeGate HR 12-9 12 Вольт 9 Ач (EP129860RUS)",
        "augmented_string": "БАТАРЕЯ ДЛЯ ИБП EXEGATE HR 12-9, 12В",
        "lost": ["9 Ач", "(EP129860RUS)"],
        "added": ["ДЛЯ ИБП"]
    },
    {
        "original_string": "Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют",
        "augmented_string": "Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют",
        "lost": [],
        "added": []
    },
    {
        "original_string": "Поддон деревянный, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)",
        "augmented_string": "Поддон деревянный 1500 кг БУ 1200х800мм",
        "lost": ["грузоподъемность", "1 сорт"],
        "added": ["БУ"]
    },
    {
        "original_string": "Картридж лазерный Хеrох 106R01277 оригинальный (двойная упаковка)",
        "augmented_string": "Картридж лазерный HEROCH 106R01277 черный ориг. ДУ",
        "lost": ["Хеrох"],
        "added": ["HEROCH", "черный"]
    }
]

examples_str = "\n".join([f"""
{i+1})
Оригинальная строка: {example["original_string"]}
Аугментированная строка: {example["augmented_string"]}
Ответ: {json.dumps({
    "lost_fragments": example["lost"],
    "added_fragments": example["added"]
}, ensure_ascii=False)}
""".strip() for i, example in enumerate(examples)])
print(examples_str)

1)
Оригинальная строка: Батарея ExeGate HR 12-9 12 Вольт 9 Ач (EP129860RUS)
Аугментированная строка: БАТАРЕЯ ДЛЯ ИБП EXEGATE HR 12-9, 12В
Ответ: {"lost_fragments": ["9 Ач", "(EP129860RUS)"], "added_fragments": ["ДЛЯ ИБП"]}
2)
Оригинальная строка: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют
Аугментированная строка: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют
Ответ: {"lost_fragments": [], "added_fragments": []}
3)
Оригинальная строка: Поддон деревянный, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)
Аугментированная строка: Поддон деревянный 1500 кг БУ 1200х800мм
Ответ: {"lost_fragments": ["грузоподъемность", "1 сорт"], "added_fragments": ["БУ"]}
4)
Оригинальная строка: Картридж лазерный Хеrох 106R01277 оригинальный (двойная упаковка)
Аугментированная строка: Картридж лазерный HEROCH 106R01277 черный ориг. ДУ
Ответ: {"lost_fragments": ["Хеrох"], "added_

In [1]:
batch = [{
    "examples": examples_str,
    "format_instructions": parser.get_format_instructions(),
    "original_string": product["title"],
    "augmented_string": res.content
} for product, res in zip(product_choices, res2)]
# print(*batch, sep="\n")

NameError: name 'product_choices' is not defined

In [134]:
compare_chain = prompt | llm

compare_results = compare_chain.batch(batch)

In [126]:
for item, compare_result in zip(batch, compare_results):
    print(item["original_string"])
    print(item["augmented_string"])
    print(compare_result.content)
    print(parser.invoke(compare_result))
    print("-------------------------------")
    

Чайник электрический Scarlett SC-EK27G95 серебристый
Эл.ЧАЙНИК SC-EK27G95 СЕР. SCARLETT
Чтобы сравнить оригинальную строку и аугментированную строку и выявить потерянные и добавленные фрагменты, следуем заданным правилам:

### Оригинальная строка:
"Чайник электрический Scarlett SC-EK27G95 серебристый"

### Аугментированная строка:
"Эл.ЧАЙНИК SC-EK27G95 СЕР. SCARLETT"

### Анализ:

1. **Сокращения**:
   - "электрический" сокращено до "Эл." – согласно правилам это не считается потерей.
   - "серебристый" сокращено до "СЕР." – согласно правилам это не считается потерей.

2. **Транслитерация и вспомогательные фрагменты**:
   - "Scarlett" транслитерировано как "SCARLETT" – добавление в данном случае отсутствует, так как это просто изменение регистра.
   - Добавлено слово "SCARLETT", которое уже присутствует в оригинальной строке в другой форме написания, и не считается потерей или добавлением по заданным правилам.

3. **Уточнения и дополнительные элементы**:
   - В оригинальной строке нет у

In [136]:
for item, compare_result in zip(batch, compare_results):
    print(item["original_string"])
    print(item["augmented_string"])
    # print(compare_result.content)
    print(parser.invoke(compare_result))
    print()

Чайник электрический Scarlett SC-EK27G95 серебристый
Эл.ЧАЙНИК SC-EK27G95 СЕР. SCARLETT
lost_fragments=['серебристый'] added_fragments=['SCARLETT']

Лоток горизонтальный для бумаг Комус Art Deco пластиковый бирюзовый
ЛОТОК ГОР. ДЛЯ БУМАГ ART DECO ПЛАСТ БИРЮЗОВЫЙ КОМУС
lost_fragments=[] added_fragments=[]

Топ для шкафа Torr (дуб каньон, 854x452x38 мм)
Топ на шкаф TORR Дуб Каньон 854x452x38мм
lost_fragments=[] added_fragments=[]

Сапоги рабочие шахтерские мужские С192Т-МТЗ ПУ резиновые черные c композитным подноском размер 42
С192Т-МТЗ САПОГИ РАБ/ШАХ МУЖ, ПУ, РЕЗИН, ЧЁРНЫЕ, КОМП. ПОДН., 42 РАЗМ.
lost_fragments=['овые'] added_fragments=['БУ']

Ложка столовая Luxstahl Lotus (кт3124) 20.4 см нержавеющая сталь (12 штук в упаковке)
Ложка столовая LUXSTAHL LOTUS КТ3124, 20.4 СМ, НСТ, 12ШТ/УПАК
lost_fragments=[] added_fragments=[]

Жидкость для снятия лака EVI professional с ацетоном 1000 мл
ЕВИ PROFESSIONAL ЖЕЛ.ДЛЯ СН.ЛАКА С АЦЕТ. 1000мл
lost_fragments=[] added_fragments=[]

Костюм рабочий зи

# Аугментация с CoT

In [9]:
class AugmentationResultResponse(BaseModel):
    answer: str = Field(..., description="Аугментированная позиция")

parser = PydanticOutputParser(pydantic_object=AugmentationResultResponse)

SYSTEM_PROMPT = """
Проведи процесс аугментации над наименованием товарной позиции.

Примеры:
{examples}

Правила аугментации:
1. **Сохранение всех деталей**  
   - При создании вариаций товарной позиции **все исходные детали должны быть сохранены**.  
   - Никакие элементы описания (например, бренд, модель, характеристики, цвет, размер) не должны быть упущены.
2. **Отсутствие добавления новых деталей**  
   - В аугментированных вариантах **запрещено добавлять новые сведения**, которые отсутствуют в исходной товарной позиции.  
   - Например, если цвет товара не указан, его нельзя добавить в описание.
3. **Не проводи транслитерацию важных деталей товарной позиции**  
   - Не аугментируй детали товарной позиции, по которым можно точно идентифицировать товарную позицию (пример: модель, компания-производитель, артикул)
4. **Не проводи частичную транслитерацию**

Подумай шаг за шагом. При рассуждении объясните, как вы соблюдаете каждое правило.

Правила для формирования ответа.
1. Рассуждение должно быть текстовым. Блоки кода в ходе рассуждения запрещены.
2. Финальный ответ должен быть представлен в виде JSON-объекта, заключённого в один блок кода.
3. Блок кода должен содержать только корректный JSON-объект без дополнительных объяснений.

После рассуждений сформируй ответ в данном формате: {format_instructions}
""".strip()

USER_PROMPT =  """
Товарная позиция: {problem_title}
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [ ]:
batch = [
    {
        "examples": examples_str,
        "problem_title": data[product_id]["title"],
        "format_instructions": parser.get_format_instructions()
    } for product_id in idx
]

In [13]:
print(prompt.invoke(batch[0]).to_messages()[0].content)

Проведи процесс аугментации над наименованием товарной позиции.

Примеры:
1)Оригинальная товарная позиция: Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)
Аугментированная товарная позиция: 4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota
2)Оригинальная товарная позиция: Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт.
Аугментированная товарная позиция: АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT
3)Оригинальная товарная позиция: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют 
Аугментированная товарная позиция: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют
4)Оригинальная товарная позиция: Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)
Аугментированная товарная позиция: Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт

Правила аугментации:
1. **Сохранение всех деталей**  
   - При создании вариаций товарной по

In [ ]:
examples = [
    {
        "original_item": "4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota",
        "clean_item": "Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)"
    },
    {
        "original_item": "АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT",
        "clean_item": "Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт."
    },
        {
        "original_item": "Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют",
        "clean_item": "Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют "
    },
    {
        "original_item": "Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт",
        "clean_item": "Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)"
    }
]

examples_str = "\n".join([f"""
{i+1})Оригинальная товарная позиция: {product["clean_item"]}
Аугментированная товарная позиция: {product['original_item']}
""".strip() for i, product in enumerate(examples)])
print(examples_str)

In [11]:
augment_chain = prompt | llm

res1 = augment_chain.batch(batch)
answer = parser.batch(res1)

In [114]:
for reasoning, product, ans in zip(res1, batch, answer):
    print(product["problem_title"])
    print(reasoning.content)
    print(ans.answer)
    print("--------------------------------")

NameError: name 'answer' is not defined

# Эксперимент с моделью парсера

In [31]:
class AugmentationResultResponse(BaseModel):
    answer: str = Field(..., description="Аугментированная позиция")
    changes: List[Tuple[str, str]] = Field(..., description="Список изменений при аугментации. Каждая пара представляет собой кортеж из двух строк. Первая строка - до изменений, вторая - после изменений. Обе строки не могут быть пустыми.")

parser = PydanticOutputParser(pydantic_object=AugmentationResultResponse)

SYSTEM_PROMPT = """
Проведи процесс аугментации над наименованием товарной позиции.

Примеры:
{examples}

Правила аугментации:
1. **Сохранение всех деталей**  
   - При создании вариаций товарной позиции **все исходные детали должны быть сохранены**.  
   - Никакие элементы наименования товарной позиции не должны быть упущены.
2. **Отсутствие добавления новых деталей**  
   - В аугментированных вариантах **запрещено добавлять новые сведения**, которые отсутствуют в исходной товарной позиции.  
   - Например, если цвет товара не указан, его нельзя добавить в описание.
3. **Не проводи транслитерацию важных деталей товарной позиции**  
   - Не аугментируй детали товарной позиции, по которым можно точно идентифицировать товарную позицию (пример: модель, компания-производитель, артикул)
4. **Не проводи частичную транслитерацию**

Подумай шаг за шагом. 

Правила для формирования ответа:
1. **Рассуждение должно быть текстовым.**  
   - Рассуждения должны быть представлены в виде сплошного текста. Блоки кода в ходе рассуждения запрещены.
   - Все логические шаги, анализы и выводы должны быть изложены четко и последовательно.
2. **Финальный ответ должен быть представлен в виде JSON-объекта, заключённого в один блок кода.**  
   - Финальный ответ должен быть корректным JSON-объектом, который заключается в один блок кода (используя три обратных апострофа ```).
   - Внутри JSON-объекта не должно быть дополнительных объяснений или текстовых комментариев.
3. **Фраза "Окончательный ответ" должна предварять блок кода с JSON-объектом.**  
   - Перед блоком кода с JSON-объектом должна быть написана фраза **"Окончательный ответ:"** (без кавычек в самой фразе).  
   - Между фразой и блоком кода не должно быть пустых строк.

Формат ответа должен соответствовать этому: {format_instructions}
""".strip()

USER_PROMPT =  """
Товарная позиция: {problem_title}
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

In [32]:
examples = [
    {
        "original_item": "4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota",
        "clean_item": "Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)"
    },
    {
        "original_item": "АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT",
        "clean_item": "Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт."
    },
        {
        "original_item": "Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют",
        "clean_item": "Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют "
    },
    {
        "original_item": "Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт",
        "clean_item": "Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)"
    },
    {
        "original_item": "Пуф складной с ящиком для хранения Куба цвет черный, иск. кожа, 370 мм, 370мм, 400мм",
        "clean_item": "Пуф Куба складной с ящиком для хранения черный (искусственная кожа, 370x370x400 мм)"
    }
]

examples_str = "\n".join([f"""
{i+1})Оригинальная товарная позиция: {product["clean_item"]}
Аугментированная товарная позиция: {product['original_item']}
""".strip() for i, product in enumerate(examples)])
print(examples_str)

batch = [
    {
        "examples": examples_str,
        "problem_title": data[product_id]["title"],
        "format_instructions": parser.get_format_instructions()
    } for product_id in idx
]
# print(*batch, sep="\n")

1)Оригинальная товарная позиция: Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)
Аугментированная товарная позиция: 4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota
2)Оригинальная товарная позиция: Абразивный круг на липучке Sunmight P500, 15 отверстий, 1 шт.
Аугментированная товарная позиция: АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT
3)Оригинальная товарная позиция: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют 
Аугментированная товарная позиция: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют
4)Оригинальная товарная позиция: Поддон деревянный БУ, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)
Аугментированная товарная позиция: Поддон деревянный БУ 1200х800мм г/п 1500кг 1 сорт
5)Оригинальная товарная позиция: Пуф Куба складной с ящиком для хранения черный (искусственная кожа, 370x370x400 мм)
Аугментированная товарная позиция: Пуф складной с 

In [33]:
augment_chain = prompt | llm

res1 = augment_chain.batch(batch)


In [15]:
import re
import time

def remove_comments_from_json(json_str):
    """
    Удаляет однострочные комментарии (начинающиеся с //) из JSON-строки.
    
    :param json_str: Исходная строка JSON с комментариями.
    :return: Строка JSON без комментариев.
    """
    # Регулярное выражение для поиска однострочных комментариев
    json_without_comments = re.sub(r'//.*', '', json_str)
    
    # Убираем лишние пробелы и пустые строки, чтобы сохранить читаемость
    json_cleaned = '\n'.join(line.strip() for line in json_without_comments.splitlines() if line.strip())
    
    return json_cleaned


def timing_decorator(func):
    """
    Декоратор для измерения времени выполнения функции.
    """
    def wrapper(*args, **kwargs):
        # Засекаем время начала выполнения функции
        start_time = time.time()
        
        # Вызываем саму функцию
        result = func(*args, **kwargs)
        
        # Засекаем время окончания выполнения функции
        end_time = time.time()
        
        # Вычисляем затраченное время
        elapsed_time = end_time - start_time
        
        # Выводим результат
        print(f"Функция '{func.__name__}' выполнилась за {elapsed_time:.6f} секунд.")
        
        # Возвращаем результат выполнения функции
        return result
    
    return wrapper

In [ ]:
print(len([1 for item in res1 if "Окончательный ответ:" in item.content]))

for item, product in zip(res1, batch):
    # print(item.content)
    if "Окончательный ответ:" in item.content:
        # print(item.content)

        split_ans_with_comments = item.content.split("Окончательный ответ:")[1]

        split_ans = remove_comments_from_json(split_ans_with_comments)
        try:
            # print(SimpleJsonOutputParser().invoke(split_ans))
            parser.invoke(split_ans)
        except Exception as e:
            print(product["problem_title"])
            print(f"Error: {e}")
    else:
        print(item.content)

100


In [34]:
for item, product in zip(res1, batch):
    # print(product["problem_title"])
    if "Окончательный ответ:" in item.content:
        # print(item.content)

        split_ans_with_comments = item.content.split("Окончательный ответ:")[1]

        split_ans = remove_comments_from_json(split_ans_with_comments)
        try:
            # print(SimpleJsonOutputParser().invoke(split_ans))
            parsed_ans = parser.invoke(split_ans)
            print(f"""| {product["problem_title"]} | {parsed_ans.answer} | {"<br>".join(str(tuple_changes) for tuple_changes in parsed_ans.changes)} |""")
        except Exception as e:
            print(product["problem_title"])
            print(f"Error: {e}")
    else:
        print(item.content)

| Мешки для мусора Luscan 200 л прозрачные (ПВД 50 мкм, 10 штук в рулоне) | прозрачные МЕШКИ ДЛЯ МУСОРА Luscan объём 200 л, материал ПВД 50 мкм, 10 шт. в рулоне | ('Мешки для мусора', 'МЕШКИ ДЛЯ МУСОРА')<br>('200 л', 'объём 200 л')<br>('прозрачные', 'прозрачные МЕШКИ')<br>('10 штук в рулоне', '10 шт. в рулоне') |
| Мармеладки в шоколаде Апельтини апельсиновые палочки 160 г | Мармеладки шоколадные Апельтини вкус апельсин, палочки 160г (0,16 кг) | ('Мармеладки в шоколаде', 'Мармеладки шоколадные')<br>('апельсиновые палочки', 'вкус апельсин, палочки')<br>('160 г', '160г (0,16 кг)') |
| Куртка рабочая летняя мужская л03-КУ с СОП синяя (размер 48-50 рост 182-188) | Куртка раб. летняя муж. л03-КУ с СОП, синяя, цвет синий, размер 48-50 см, рост 182-188 см | ('Куртка рабочая летняя мужская', 'Куртка раб. летняя муж.')<br>('размер 48-50', 'размер 48-50 см')<br>('рост 182-188', 'рост 182-188 см')<br>('синяя', 'синяя, цвет синий') |
| МФУ лазерное Ricoh MP 2014AD (912356/417378) | МФУ лазерное RI

In [26]:
import datetime

start = datetime.datetime.now()
answer = parser.batch(res1)
print(datetime.datetime.now() - start)

start = datetime.datetime.now()
for res in res1:
    parser.invoke("""Формируем JSON-объект:

```json
{
  "answer": "Стол прям. Канц 1400х600х750мм дуб молочный"
}
```
                  """)
print(datetime.datetime.now() - start)


0:00:00.125812
0:00:00.004996
